In [135]:
import numpy as np
import math

real = int | float
Real = {int, float}

In [136]:
def f(x):
    return x**2

a, b, n = 1, 10, 10

In [137]:
x_uniform = np.linspace(a, b, n)
y_uniform = f(x_uniform)
print("x_uniform =", x_uniform)
print("y_uniform =", y_uniform)

x_uniform = [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
y_uniform = [  1.   4.   9.  16.  25.  36.  49.  64.  81. 100.]


In [138]:
def tchebychev_points(a, b, n):
    return (a + b)/2 - (b - a)/2 * np.array([math.cos(math.pi*(2*i+1)/(2*n)) for i in range(n)])

x_tchebychev = tchebychev_points(a, b, n)
y_tchebychev = f(x_tchebychev)
print("x_tchebychev =", x_tchebychev)
print("y_tchebychev =", y_tchebychev)

x_tchebychev = [1.05540247 1.49047064 2.31801948 3.45704275 4.79604491 6.20395509
 7.54295725 8.68198052 9.50952936 9.94459753]
y_tchebychev = [ 1.11387437  2.22150273  5.37321433 11.95114458 23.00204675 38.48905879
 56.89620406 75.37678567 90.43114863 98.89502009]


In [139]:
class NewtonPolynomial:
    def __init__(self, x, y, name='p'):
        self.x = np.array(x, dtype=float)
        self.y = np.array(y, dtype=float)
        self.coefficients = self.newton_interpol_coefs(self.x, self.y)
        self.name = name

    def newton_interpol_coefs(self, x, y):
        n = len(x)
        coefficients = y.copy()
        step = 1
        for j in range(1, n):
            coefficients[j:n] = (coefficients[j:n] - coefficients[j-1:n-1]) / (x[j:n] - x[j-step:n-step])
            step += 1
        return coefficients

    def horner_eval(self, x0):
        n = len(self.coefficients)
        result = self.coefficients[-1]
        for i in range(n - 2, -1, -1):
            result = result * (x0 - self.x[i]) + self.coefficients[i]
        return result

    def __str__(self):
        nb_dec_digits = 3
        terms = [f"{self.coefficients[0]}"] if self.coefficients[0] != 0 else []
        term = ""
        for i in range(0, len(self.coefficients)-1):
            if self.coefficients[i+1] == 0:
                continue
            term += f"(x - {round(self.x[i], nb_dec_digits)})" if self.x[i] != 0 else "x"
            terms.append(f"{round(self.coefficients[i+1], nb_dec_digits)} * " + term)
        return self.name + "(x) = " + " + ".join(terms)

x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
y = [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
print("x =", x)
print("y =", y)
polynomial = NewtonPolynomial(x, y)
print(polynomial)

x = 7
value = polynomial.horner_eval(7)
print(f"p({x}) = {value}")

x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
y = [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
p(x) = 1.0 + 3.0 * (x - 1.0) + 1.0 * (x - 1.0)(x - 2.0)
p(7) = 49.0


In [140]:
uniform_lagrange_poly_f = NewtonPolynomial(x_uniform, y_uniform)

x0 = 7
print("x_uniform =", x_uniform)
print("y_uniform =", y_uniform)
print(uniform_lagrange_poly_f.horner_eval(x0))


x_uniform = [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
y_uniform = [  1.   4.   9.  16.  25.  36.  49.  64.  81. 100.]
49.0


In [141]:
tchebychev_lagrange_poly_f = NewtonPolynomial(x_tchebychev, y_tchebychev)

print("x_tchebychev =", x_tchebychev)
print("y_tchebychev =", y_tchebychev)
print(tchebychev_lagrange_poly_f.horner_eval(x0))

x_tchebychev = [1.05540247 1.49047064 2.31801948 3.45704275 4.79604491 6.20395509
 7.54295725 8.68198052 9.50952936 9.94459753]
y_tchebychev = [ 1.11387437  2.22150273  5.37321433 11.95114458 23.00204675 38.48905879
 56.89620406 75.37678567 90.43114863 98.89502009]
49.0


In [22]:
x = np.array([1, 2, 4, 7], dtype=float)
y = np.array([1, 4, 2, 8])
z = x[1:3] / y[1:3]
x[1:3] = z[:]
print(z, x)

[0.5 2. ] [1.  0.5 2.  7. ]


In [ ]:
class LagrangeLinearPolynomial:
    def __init__(self, x, y):
        self.x = np.array(x, dtype=float)
        self.y = np.array(y, dtype=float)
        self.name = 'p'
        self.polynomials = [NewtonPolynomial(self.x[i:i+2], self.y[i:i+2]) for i in range(len(self.x))]
        self.coefficients = self.lagrange_interpol_coefs(self.x, self.y)

    def eval(self, x0):
        before_x0 = 0
        after_x0 = 0
        for i in range(1, len(self.polynomials)):
            if self.x[i] > self.x[before_x0] and x0 < self.x[i]:
                before_x0 = i
            if self.x[i] < self.x[after_x0] and x0 > self.x[i]:
                after_x0 = i
        if before_x0 == 0 and self.x[0] > x0:
            raise ValueError(f"!{x0} est avant la plage des abscisses")
        if after_x0 == 0 and self.x[0] < x0:
            raise ValueError(f"!{x0} est apres la plage des abscisses")
        return self.polynomials[before_x0].horner_eval(x0)

In [ ]:
class TaylorPolynomial:
    def __init__(self, x, y, name='p'):
        self.x = np.array(x, dtype=float)
        self.y = np.array(y, dtype=float)
        self.coefficients = self.taylor_interpol_coefs(self.x, self.y)
        self.name = name

    def taylor_interpol_coefs(self, x, y):
        n = len(x)
        coefficients = y.copy()
        step = 1
        for j in range(1, n):
            coefficients[j:n] = (coefficients[j:n] - coefficients[j-1:n-1]) / (x[j:n] - x[j-step:n-step])
            step += 1
        return coefficients

    def horner_eval(self, x0):
        n = len(self.coefficients)
        result = self.coefficients[-1]
        for i in range(n - 2, -1, -1):
            result = result * (x0 - self.x[i]) + self.coefficients[i]
        return result

    def __str__(self):
        nb_dec_digits = 3
        terms = [f"{self.coefficients[0]}"] if self.coefficients[0] != 0 else []
        term = ""
        for i in range(0, len(self.coefficients)-1):
            if self.coefficients[i+1] == 0:
                continue
            term += f"(x - {round(self.x[i], nb_dec_digits)})" if self.x[i] != 0 else "x"
            terms.append(f"{round(self.coefficients[i+1], nb_dec_digits)} * " + term)
        return self.name + "(x) = " + " + ".join(terms)

x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
y = [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
print("x =", x)
print("y =", y)
polynomial = NewtonPolynomial(x, y)
print(polynomial)

x = 7
value = polynomial.horner_eval(7)
print(f"p({x}) = {value}")

In [ ]:


class SplineCubicPolynomial:
    def __init__(self, x, y):
        self.x = np.array(x, dtype=float)
        self.y = np.array(y, dtype=float)
        self.name = 'p'
        self.polynomials = [NewtonPolynomial(self.x[i:i+2], self.y[i:i+2]) for i in range(len(self.x))]
        self.coefficients = self.lagrange_interpol_coefs(self.x, self.y)

    def eval(self, x0):
        before_x0 = 0
        after_x0 = 0
        for i in range(1, len(self.polynomials)):
            if self.x[i] > self.x[before_x0] and x0 < self.x[i]:
                before_x0 = i
            if self.x[i] < self.x[after_x0] and x0 > self.x[i]:
                after_x0 = i
        if before_x0 == 0 and self.x[0] > x0:
            raise ValueError(f"!{x0} est avant la plage des abscisses")
        if after_x0 == 0 and self.x[0] < x0:
            raise ValueError(f"!{x0} est apres la plage des abscisses")
        return self.polynomials[before_x0].horner_eval(x0)